# 本程序用于观察数据的基本特征、计算单品日销量、计算各种类蔬菜日销量、计算各种类蔬菜平均批发价

In [1]:
import numpy as np
import pandas as pd

In [4]:
# 为了方便，将附件的文件名和各附件内不同列名进行了修改（转换为英文）;注意：读取数据时要设置文件路径
df = pd.read_excel("1.xlsx")
df1=pd.read_excel("2.xlsx")
df2=pd.read_excel("3.xlsx")
df3=pd.read_excel("4_1.xlsx", index_col=0)

In [20]:
## 将附件一的蔬菜种类转换为数字，附件二的销售类型与打折转换为数字（可以不进行，如果要做这一步就要先把附录的列名进行修改）

# columns=["category_name"]
# mapping_dict = {}
# for column_name in columns:
#     unique_values = df[column_name].unique()
#     value_to_number = {value: number for number, value in enumerate(unique_values)}
#     mapping_dict[column_name] = value_to_number

# # Replace values in all columns with numbers
# for column_name in columns:
#     df[column_name] = df[column_name].replace(mapping_dict[column_name])

# columns=["type","is_discount"]
# mapping_dict = {}
# for column_name in columns:
#     unique_values = df1[column_name].unique()
#     value_to_number = {value: number for number, value in enumerate(unique_values)}
#     mapping_dict[column_name] = value_to_number

# # Replace values in all columns with numbers
# for column_name in columns:
#     df1[column_name] = df1[column_name].replace(mapping_dict[column_name])

## 观察数据的基本特征

In [5]:
unique_to_df1 = df1[~df1['product_code'].isin(df2['product_code'])]
unique_to_df2 = df2[~df2['product_code'].isin(df1['product_code'])]

date_unique_to_df1 = df1[~df1['sale_date'].isin(df2['sale_date'])]
date_unique_to_df2 = df2[~df2['sale_date'].isin(df1['sale_date'])]
for col in unique_to_df2['product_code'].unique():
    print(df[df['product_code'] == col])

date_unique_to_df2['sale_date'].unique()

        product_code product_name  category_code  category_name
198  102900011011782    虫草花(盒)(1)     1011010801              5
       product_code product_name  category_code  category_name
20  102900005116776         本地菠菜     1011010101              0
        product_code product_name  category_code  category_name
163  102900011032145      芜湖青椒(份)     1011010504              4
        product_code product_name  category_code  category_name
105  102900005116042            藕     1011010402              2
        product_code product_name  category_code  category_name
150  102900011023648      芜湖青椒(2)     1011010504              4


<DatetimeArray>
['2022-11-02 00:00:00', '2022-11-04 00:00:00', '2022-11-30 00:00:00',
 '2022-12-01 00:00:00', '2022-12-02 00:00:00', '2022-12-03 00:00:00']
Length: 6, dtype: datetime64[ns]

## 计算单品日销量

In [6]:
# 统计单品的日销量（缺失6天的数据）
date_unique = df1['sale_date'].unique()
product_unique = df1['product_code'].unique()
total_sales = pd.DataFrame(0, index=date_unique, columns=product_unique)  # 用来统计单品日销量
for index, row in df1.iterrows():
    saled = row['type']
    date = row['sale_date']
    kind = row['product_code']
    volume = row['volume']
    unit_price = row['unit_price']
    total_sales.at[date, kind] += volume


### 填补单品日销量缺失数据

In [7]:
# 对于单品日销量数据缺失的6天数据，由于数据缺失都发生在2022年，于是我们采取用前两年同天的数据进行填补。
# （如果某类单品在前两年同一天均有出售，我们便将今年的销售量取为前两年均值）
dates=['2022-11-02 00:00:00', '2022-11-04 00:00:00', '2022-11-30 00:00:00',
 '2022-12-01 00:00:00', '2022-12-02 00:00:00', '2022-12-03 00:00:00']
dates_datetime = pd.to_datetime(dates)

for date in dates_datetime:
    date1 = date - pd.DateOffset(years=1)
    date2 = date - pd.DateOffset(years=2)
    for col in total_sales.columns:
        if total_sales.loc[date2][col] != 0 and total_sales.loc[date1][col] != 0:
            total_sales.loc[date, col]=(total_sales.loc[date2][col]+total_sales.loc[date1][col])/2
            
total_sales=total_sales.sort_index().fillna(0)
total_sales.to_excel('蔬菜单品日销量.xlsx', index=True)

## 计算各种类蔬菜日销量

In [8]:
# 统计各类蔬菜的日销量
date_unique = total_sales.index.unique()
product_unique = df['category_name'].unique()
cate_sale = pd.DataFrame(0, index=date_unique, columns=product_unique)

for index, row in total_sales.iterrows():
    for kind in df['category_name'].unique():
        for col in total_sales.columns:
            if col in df[df['category_name']==kind]['product_code'].values:
                cate_sale.at[index, kind] += row[col]

name = ['花叶类', '花菜类', '水生根茎类', '茄类', '辣椒类', '食用菌']
cate_sale.columns = name
cate_sale.to_excel('蔬菜品类日销量.xlsx', index=True)

## 计算各种类蔬菜平均批发价

In [9]:
# 将附件1与附件三按照单品编号进行合并，使用合并的数据计算各品类日均批发价
whole_price = df2.merge(df, on=['product_code', 'product_code'], how='inner')
del whole_price['category_code']
del whole_price['product_name']
whole_price

,sale_date,product_code,wholesale_price,category_name
0,2020-07-01,102900005115762,3.88,0
1,2020-07-02,102900005115762,3.93,0
2,2020-07-03,102900005115762,3.57,0
3,2020-07-05,102900005115762,3.48,0
4,2020-07-06,102900005115762,3.35,0
...,...,...,...,...
55977,2023-06-21,102900011036686,1.37,0
55978,2023-06-23,102900011036686,1.22,0
55979,2023-06-27,102900011036686,1.46,0
55980,2023-06-29,102900011036686,1.71,0


In [12]:
# 统计各类蔬菜的批发价;也可以使用Excel进行处理，本文采用Excel处理
# date_unique = total_sales.index.unique()
# product_unique = df['category_name'].unique()
# price = pd.DataFrame(0, index=date_unique, columns=product_unique)
# for kind in df['category_name'].unique():
#     # print(kind)
#     # print('\n')
#     for index, row in whole_price.iterrows():
#         # print(whole_price['category_name'].values==kind)
#         # break
#         if whole_price['category_name'].values == kind:
#             price.at[index, kind] += row['wholesale_price']
# name = ['花叶类', '花菜类', '水生根茎类', '茄类', '辣椒类', '食用菌']
# price.columns = name
# price.to_excel('品类日批发价.xlsx', index=True)